In [1]:
import pandas as pd
import altair as alt


alt.data_transformers.disable_max_rows()

url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"

columns = [
    "datetime", "city", "state", "country", "shape", "duration_seconds",
    "duration_reported", "summary", "date_posted", "latitude", "longitude"
]

ufo = pd.read_csv(url, names=columns, header=0)  

ufo['datetime'] = pd.to_datetime(ufo['datetime'], errors='coerce')

ufo['year'] = ufo['datetime'].dt.year

ufo = ufo.dropna(subset=['year'])

ufo.head()

,datetime,city,state,country,shape,duration_seconds,duration_reported,summary,date_posted,latitude,longitude,year
0,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,1949
1,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,1955
2,1956-10-10 21:00:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,1956
3,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,1960
4,1961-10-10 19:00:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,1961


In [2]:
ufo['datetime'] = pd.to_datetime(ufo['datetime'], errors='coerce')
ufo['year'] = ufo['datetime'].dt.year
year_counts = ufo[ufo['year'] > 1970]['year'].value_counts().sort_index().reset_index()
year_counts.columns = ['year', 'count']

chart1 = alt.Chart(year_counts).mark_line().encode(
    x='year:O',
    y='count:Q'
).properties(
    title='UFO Sightings per Year (after 1970)',
    width=500
)


In [3]:
ufo_filtered = ufo.dropna(subset=['shape', 'state'])

state_dropdown = alt.binding_select(options=sorted(ufo_filtered['state'].unique()))
state_select = alt.selection_single(fields=['state'], bind=state_dropdown, name="Select")

chart2 = alt.Chart(ufo_filtered).mark_bar().encode(
    x=alt.X('shape:N', title='UFO Shape'),
    y=alt.Y('count():Q', title='Number of Sightings'),
    color=alt.Color('shape:N', legend=None),
    tooltip=['shape', 'count()']
).transform_filter(
    state_select
).properties(
    width=600,
    height=400,
    title='UFO Shapes by State (Interactive)'
).add_params(
    state_select
)



/var/folders/4y/gfvp3jps5710ysvdz003nhnc0000gn/T/ipykernel_28381/1451603077.py:4: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  state_select = alt.selection_single(fields=['state'], bind=state_dropdown, name="Select")


In [4]:

chart1.save('ufo_per_year.json')
chart2.save('ufo_by_shape_interactive.json')
